decreasing overfitting using dropout regulatization

In [60]:
import numpy as np
import pandas as pd

In [61]:
df = pd.read_csv(r"C:\Users\91779\Downloads\sonar_dataset.csv",header=None) #header = None, becausre it does'nt have columns name
df.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [62]:
df.isnull().sum() #no null value in any column

0     0
1     0
2     0
3     0
4     0
     ..
56    0
57    0
58    0
59    0
60    0
Length: 61, dtype: int64

In [63]:
df[60].value_counts()

M    111
R     97
Name: 60, dtype: int64

In [64]:
x = df.drop([60],axis =1)
y = df[60]
y.head()

0    R
1    R
2    R
3    R
4    R
Name: 60, dtype: object

In [65]:
# one hot encoding
y = pd.get_dummies(y,drop_first=True)
y.sample(5)

,R
180,0
24,1
40,1
88,1
10,1


In [66]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=1)

In [67]:
x_train.shape,x_test.shape

((156, 60), (52, 60))

In [68]:
import tensorflow as tf
from tensorflow import keras

In [69]:
model = keras.Sequential([
    keras.layers.Dense(60,input_dim=60,activation='relu'),
    keras.layers.Dense(30,activation='relu'),
    keras.layers.Dense(15,activation='relu'),
    keras.layers.Dense(1,activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(x_train,y_train,epochs=100,batch_size=8)

Epoch 1/100
20/20 [==============================] - 1s 3ms/step - loss: 0.6892 - accuracy: 0.5256
Epoch 2/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6663 - accuracy: 0.6218
Epoch 3/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6511 - accuracy: 0.6538
Epoch 4/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6250 - accuracy: 0.7244
Epoch 5/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6039 - accuracy: 0.7115
Epoch 6/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5794 - accuracy: 0.7436
Epoch 7/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5426 - accuracy: 0.7756
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5129 - accuracy: 0.7564
Epoch 9/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4721 - accuracy: 0.7949
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4517 - accuracy: 0.8205
Epoch 11/

In [70]:
model.evaluate(x_test,y_test)

2/2 [==============================] - 0s 8ms/step - loss: 0.9174 - accuracy: 0.7692


[0.9174283742904663, 0.7692307829856873]

In [71]:
y_pred = model.predict(x_test).reshape(-1)
print(y_pred[:5])

#rounding to integer
y_pred = np.round(y_pred)
print(y_pred[:5])

2/2 [==============================] - 0s 8ms/step
[1.1799693e-09 4.9431187e-01 6.1099756e-01 3.0159731e-06 9.9999928e-01]
[0. 0. 1. 0. 1.]


In [72]:
y_test[:5]

,R
186,0
155,0
165,0
200,0
58,1


In [73]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.71      0.93      0.81        27
           1       0.88      0.60      0.71        25

    accuracy                           0.77        52
   macro avg       0.80      0.76      0.76        52
weighted avg       0.80      0.77      0.76        52



In [74]:
# noe using dopout-regu;arization
model_drop = keras.Sequential([
    keras.layers.Dense(60,input_dim=60,activation='relu'),
    keras.layers.Dropout(0.5),#using 50% dropout in dropout layer
    keras.layers.Dense(30,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(15,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1,activation='sigmoid')
])

model_drop.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model_drop.fit(x_train,y_train,epochs=100,batch_size=8)

Epoch 1/100
20/20 [==============================] - 1s 2ms/step - loss: 0.7479 - accuracy: 0.4936
Epoch 2/100
20/20 [==============================] - 0s 3ms/step - loss: 0.7260 - accuracy: 0.5577
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6710 - accuracy: 0.5321
Epoch 4/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6844 - accuracy: 0.5513
Epoch 5/100
20/20 [==============================] - 0s 2ms/step - loss: 0.7211 - accuracy: 0.5000
Epoch 6/100
20/20 [==============================] - 0s 2ms/step - loss: 0.7096 - accuracy: 0.5064
Epoch 7/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6747 - accuracy: 0.5641
Epoch 8/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6768 - accuracy: 0.5385
Epoch 9/100
20/20 [==============================] - 0s 2ms/step - loss: 0.7047 - accuracy: 0.5385
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6782 - accuracy: 0.6026
Epoch 11/

In [76]:
model_drop.evaluate(x_test,y_test)

2/2 [==============================] - 0s 9ms/step - loss: 0.3879 - accuracy: 0.7885


[0.3879300653934479, 0.7884615659713745]

In [77]:
y_pred_drop = np.round(model_drop.predict(x_test).reshape(-1))

2/2 [==============================] - 0s 0s/step


In [78]:
y_pred_drop[:5]

array([0., 1., 1., 0., 1.], dtype=float32)

In [79]:
print(classification_report(y_test,y_pred_drop))

              precision    recall  f1-score   support

           0       0.77      0.85      0.81        27
           1       0.82      0.72      0.77        25

    accuracy                           0.79        52
   macro avg       0.79      0.79      0.79        52
weighted avg       0.79      0.79      0.79        52

